In [30]:
import numpy as np
import sys
import os.path
import glob
import random
from collections import OrderedDict
from scipy.stats import lognorm, norm
from matplotlib import pyplot as plt
import mplhep as hep 
import pyhf
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
plt.style.use('loca_v_1_5/plot_style.txt')
from loca_v_1_5.loca import *
from run_scripts import *
import time
plt.style.use(hep.style.CMS)

In [17]:
def mu_mu_CMS_search(lhco_file, m_min=0.0):

    process='p p --> µ+ µ-'

    search_description='''
    Recast of CMS search CMS PAS EXO-19-019:
    Search for a narrow resonance in high-mass dilepton final
    states in proton-proton collisions using 140 fb^−1
    of data at sqrt(s)=13 TeV'''
    
    #...Pre-cuts:
    basic_cuts = {                    
            'e'     : [35.0, 1e+10 ,  2.5],  # [pt_min, pt_max, abs(eta_max)] 
            'mu'    : [53.0, 1e+10 ,   2.4],
            'tau'   : [0.0 , 1e+10 ,  1e+10],
            'jet'   : [0.0 , 1e+10 ,  1e+10],
            'bjet'  : [0.0 , 1e+10 ,  1e+10],
            'met'   : [0.0 , 1e+10 ,  1e+10],
            'photon': [0.0 , 1e+10 ,  1e+10] 
             }

    signals=[]; effs=[]; xsecs=[]

    #...Signal region defintion:
    
    Category_muon = category(
                'nmuons >= 2',
                'OS µµ',
                'Mµµ > '+str(m_min))
        
    #...Observables:

    bins=[120., 129.955, 140.735, 152.41, 165.054, 178.746, 193.574, 209.632, 
          227.022, 245.855, 266.25, 288.337, 312.257, 338.16, 366.213, 396.592,
          429.492, 465.121, 503.706, 545.491, 590.743, 639.749, 692.82, 750.294, 
          812.536, 879.94, 952.937, 1031.99, 1117.6, 1210.31, 1310.71, 1419.44,
          1537.2, 1664.72, 1802.82, 1952.37, 2114.33, 2289.73, 2479.67, 2685.38,
          2908.15, 3149.4, 3410.66, 3693.59, 4000.]
    
    Mµµ=spectrum(name='Mµµ')
    
    #...Apply cuts:
    
    lhco=open_lhco(file=lhco_file, cuts=basic_cuts)

    with lhco as events:   
        for N, event in events:

            #....start cuts

            Category_muon.start_cutflow(name='dimuon')

            if event.nleps>=2:                    
                µ1=event.lepton_leading
                µ2=event.lepton_subleading

                if (µ1.is_muon and µ2.is_muon):
                    Category_muon.apply_cut('nmuons >= 2')

                    if µ1.charge==-µ2.charge:
                        Category_muon.apply_cut('OS µµ')

                        m=µ1.invM(µ2) 

                        if m > m_min:
                            Category_muon.apply_cut('Mµµ > '+str(m_min))
                            Mµµ.extract_spectrum(m) 
                            lhco.parameters['signal_category']=Category_muon

        lhco.parameters[Mµµ.name]=Mµµ
        lhco.parameters['Mµµ_bins']=bins
        lhco.parameters['effs']=Mµµ.bin_data(bins)/float(lhco.parameters['nevents'])

    return lhco.parameters


In [155]:
def extract_results(data_files):

    FF={'FS':'scalar','FT':'tensor','FST':'scalar-tensor',
        'FVLL':'vector LL','FVLR':'vector LR','FD':'dipole','FV':'SM'}      
    
    kernel={}
    
    for lhco in glob.glob(data_files, recursive=True):        
        print('analyzing: '+lhco)
        res=mu_mu_CMS_search(lhco, m_min=150)
        info=res['run_tag'].split('_') 
        kernel[res['run_tag']]={}        
        kernel[res['run_tag']]['Form_Factor']=FF[info[0]]
        if info[1]=='SM':
            kernel[res['run_tag']]['FF_coeff']=None
        else:
            kernel[res['run_tag']]['FF_coeff']=list(info[1])
        kernel[res['run_tag']]['qq~']=info[2]+'~'
        kernel[res['run_tag']]['ll~']=info[3]
        kernel[res['run_tag']]['type']=info[4]+'*'+info[5]
        kernel[res['run_tag']]['mll_bin']=[float(info[-2]),float(info[-1])]
        kernel[res['run_tag']]['results']=res
            
    return kernel

def kernel_matrix(kernel,bins,output):  
    
    def get_bins(bins):
        d={}
        for i,b in enumerate(bins[:-1]):
            I=str([float(b),float(bins[i+1])])
            d[I]=i
        return d
    runs=[] 
    info={}
    for run in kernel.keys():
        x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
        if x not in runs:
            runs.append(x)
            info[x]=kernel[run]
    dic={}
    for key in runs:
        dic[key]={}
    
    bin_number=get_bins(bins)
    
    for r in runs:
        for run in kernel.keys():
            x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
            b=str(kernel[run]['mll_bin'])
            eff=kernel[run]['results']['effs']
            if (kernel[run]['type']=='A*Reg' or
               kernel[run]['type']=='Z*Reg' or
               kernel[run]['type']=='W*Reg'):
                xsec=round(kernel[run]['results']['xsec'],5)
            else:
                xsec=round(2*kernel[run]['results']['xsec'],5)
            pre=[str(xsec),
                 str(kernel[run]['mll_bin'][0]),
                 str(kernel[run]['mll_bin'][1])]
            
            eff_str=''
            
            for y in eff: eff_str+=str(y)+'\t'            
            dic[x][bin_number[b]]='  '.join(pre+[eff_str]) 
    
#     failed_dir=make_dir(output+'/failed_runs_'+, overwrite=True)

    for d in dic.keys():
        file=open(output+'/'+d+'.dat','w')
        file.write('#===============\n')
        file.write('# ll~ : '+ info[d]['ll~'] +'\n')
        file.write('# qq~ : '+ info[d]['qq~'] +'\n')
        ff=info[d]['Form_Factor'].split()
        file.write('# FF  : '+ ff[0] +'\n')
        
        # XY:
        if len(ff)==1:    
            file.write('# XY  : LL LR RL RR\n')
        else:
            if info[d]['Form_Factor'].split()[1]=='LL':
                file.write('# XY  : LL RR  \n')
            elif info[d]['Form_Factor'].split()[1]=='LR':
                file.write('# XY  : LR RL  \n')  
                
        if info[d]['FF_coeff']:
            coeff=info[d]['FF_coeff'][0].replace("'", "")+' '+info[d]['FF_coeff'][1].replace("'", "")
        else:
            coeff='None'
        file.write('# coef : '+coeff+'\n')
        file.write('# type : '+info[d]['type']+'\n')
        file.write('# Nevs : '+ str(info[d]['results']['nevents'])+'\n')
        file.write('#===============\n'  )
        file.write('#\n'  )
        file.write('# XSEC\tBIN_MIN\tBIN_MAX \t Kij \n')
        file.write('#\n'  )
                
        for i in list(bin_number.values()):
            try:
                file.write(dic[d][i]+'\n')
            except KeyError:
                inv_bin_number = {v: k for k, v in bin_number.items()}
                print('Error -->',d,':',inv_bin_number[i])
                file.write((str(None)+'\t')*len(bin_number)+'\n')
                
                # generated failed run:
                
                x=d.split('_')
                xdic={'mumu':'mu_mu', 'tata':'ta_ta', 'ee':'e_e', 
                      'muvm':'mu_vm', 'tavt':'ta_vt', 'eve':'e_ve',
                      'eta':'e_ta', 'tae':'ta_e','muta':'mu_ta',
                      'tamu':'ta_mu','emu':'e_mu','mue':'mu_e'}    
                bin_i=str(bins[i])+'_'+str(bins[i+1])
                x=['PROC']+x+[bin_i]
                qq=x[3]
                x=x[:3]+list(x[3][0])+x[4:]
                run_path='run_scripts/scripts_'+xdic[x[4]]+'/scripts_SMEFT_'+xdic[x[4]]+'_'+bin_i+'/'+'_'.join(x)
                
                new_file=open(output+'/scripts_failed_runs/'+'_'.join(x),'w')
                
                with open(run_path) as f:
                    for line in f:  
                        if '# run_'+qq in line:
                            new_file.write(line)
                        elif '# run_' in line:
                            continue
                        else:
                            new_file.write(line)
                new_file.close()
        file.close()
 

In [156]:
# t0 = time.time()

# FS_kernel  = extract_results(data_files='lhco_files/FS_lhco/*.gz')
# FST_kernel = extract_results(data_files='lhco_files/FST_lhco/*.gz')
# FT_kernel  = extract_results(data_files='lhco_files/FT_lhco/*.gz')
# FVLL_kernel  = extract_results(data_files='lhco_files/FVLL_lhco/*.gz')
# FVLR_kernel  = extract_results(data_files='lhco_files/FVLR_lhco/*.gz')
# FD_kernel  = extract_results(data_files='lhco_files/FD_lhco/*.gz')
# FV_kernel  = extract_results(data_files='lhco_files/FV_lhco/*.gz')

# t1 = time.time()

# print('time =',(t1-t0)/3600.)

In [157]:
output=make_dir('Kernel_effs', overwrite=True)
make_dir('Kernel_effs/scripts_failed_runs', overwrite=True)
bins=[200,300,400,500,600,700,800,900,1000,1200,1400,1600,1800,2000,2250,2500,2750,3000,3500,4000,15000]
kernel_matrix(FS_kernel,bins,output)
kernel_matrix(FT_kernel,bins,output)
kernel_matrix(FST_kernel,bins,output)
kernel_matrix(FVLL_kernel,bins,output)
kernel_matrix(FVLR_kernel,bins,output)
kernel_matrix(FD_kernel,bins,output)
kernel_matrix(FV_kernel,bins,output)

Error --> FVLL_00_db_mumu_Reg_Reg : [200.0, 300.0]
Error --> FVLL_00_uc_mumu_Reg_Reg : [200.0, 300.0]
Error --> FVLR_00_uc_mumu_Reg_Reg : [300.0, 400.0]
Error --> FVLR_00_uc_mumu_Reg_Reg : [3000.0, 3500.0]
Error --> FVLR_00_uu_mumu_Reg_Reg : [3000.0, 3500.0]
Error --> FVLR_00_uu_mumu_Z_Reg : [4000.0, 15000.0]
Error --> FVLR_00_db_mumu_Reg_Reg : [300.0, 400.0]
Error --> FVLR_00_ds_mumu_Reg_Reg : [300.0, 400.0]
Error --> FVLR_00_ss_mumu_Z_Reg : [4000.0, 15000.0]
Error --> FVLR_00_sb_mumu_Reg_Reg : [300.0, 400.0]
Error --> FVLR_00_sb_mumu_Reg_Reg : [2500.0, 2750.0]
Error --> FVLR_00_dd_mumu_Z_Reg : [3000.0, 3500.0]
Error --> FVLR_00_ss_mumu_Reg_Reg : [2500.0, 2750.0]
Error --> FV_SM_uu_mumu_Z_Z : [1000.0, 1200.0]
